Script to calculate the expected PV performance without weather

Import Packages

In [160]:
import numpy as np              
import matplotlib.pyplot as plt
import PyQt6
import datetime
import ephem
import pandas as pd

Functions

In [161]:
def calculate_sun_position(latitude, longitude, date):
    # Calculate the sun's position for given coordinates and date
    observer = ephem.Observer()
    observer.lat = str(latitude)
    observer.lon = str(longitude)
    observer.date = date

    sun = ephem.Sun()
    sun.compute(observer)

    azimuth = np.rad2deg(sun.az)
    elevation = np.rad2deg(sun.alt)
    return azimuth, elevation

def calculate_delta_angle(sun_azimut, panel_orientation, sun_elevation, panel_tilt):
    # Calculate the angle between the sun and the solar panel
    # Convert to radians
    sun_azimut = np.deg2rad(sun_azimut)
    panel_orientation = np.deg2rad(panel_orientation)
    sun_elevation = np.deg2rad(sun_elevation)
    panel_tilt = np.deg2rad(panel_tilt)
    # Convert to spherical coordinates, r = 1
    theta_sun = np.pi / 2 - sun_elevation
    theta_panel = np.pi / 2 - panel_tilt
    phi_sun = sun_azimut
    phi_panel = panel_orientation
    # Calculate dot product of the two vectors
    dot_product = np.sin(theta_sun) * np.sin(theta_panel) * np.cos(
        phi_sun - phi_panel
    ) + np.cos(theta_sun) * np.cos(theta_panel)
    # Calculate angle in degrees
    angle = np.rad2deg(np.arccos(dot_product))
    return angle

def create_timestamps(date, time_step):
    # Create an array of timestamps for one day
    start_time = datetime.datetime(date.year, date.month, date.day)
    end_time = start_time + datetime.timedelta(days=1)
    num_steps = int((end_time - start_time) / time_step)
    timestamps = np.array([start_time + i * time_step for i in range(num_steps)])
    return timestamps

def get_sun_trajectory(latitude, longitude, timestamps):
    # Calculate the sun's position for each timestamp
    sun_azimuths = np.empty(len(timestamps))
    sun_elevations = np.empty(len(timestamps))
    for i, timestamp in enumerate(timestamps):
        azimuth, elevation = calculate_sun_position(latitude, longitude, timestamp)
        sun_azimuths[i] = azimuth
        sun_elevations[i] = elevation
    return sun_azimuths, sun_elevations

def limit_to_daylieght(azimuths, elevations, timestamps):
    # Limit the data to daylight hours
    daylight_mask = elevations >= 0
    azimuths = azimuths[daylight_mask]
    elevations = elevations[daylight_mask]
    timestamps = timestamps[daylight_mask]
    return azimuths, elevations, timestamps

def get_pv_sun_angle(azimuths, elevations, panel_orientation, panel_tilt):
    # Calculate the angle between the sun and the solar panel for each timestep
    angles = np.empty((len(panel_orientation), len(azimuths)))
    for num_panel in range(len(panel_orientation)):
        for i in range(len(azimuths)):
            angles[num_panel, i] = calculate_delta_angle(
                azimuths[i],
                panel_orientation[num_panel],
                elevations[i],
                panel_tilt[num_panel],
            )
    return angles

def calc_irradiance_scaling(elevations):
    # Calculate the irradiance scaling for each elevation
    irradiance = np.zeros(len(elevations))
    for i in range(len(elevations)):
        if elevations[i] > 0:
            irradiance[i] = np.sin(np.deg2rad(elevations[i]))
    return irradiance

def calc_power_ratio(angles, peak_power):
    # Calculate the power ratio for each angle with respect to the peak power
    power_ratio = np.zeros(len(angles[0]))
    for panel in range(len(angles)):
        for i in range(len(power_ratio)):
            if angles[panel][i] < 90:
                power_ratio[i] += (np.cos(np.deg2rad(angles[panel][i])) 
                                   * peak_power[panel])
    return power_ratio


User input

In [162]:
# Coordinates in degrees
latitude = 48.8
longitude = 9

# Panel orientation, tilt and peak power
orientation = np.array([96, 276]) # degrees, north = 0, east = 90, ...
tilt = np.array([30, 45]) # degrees, 0 = horizontal, 90 = vertical
peak_power = np.array([3320, 4980]) # Watts peak

# Date and stepsize
date = (datetime.datetime.now() # Current date
        + datetime.timedelta(weeks=0, days=0)) # Add weeks and days 
time_step = datetime.timedelta(minutes=1, seconds=0) # Time steps for the simulation


Run script

In [163]:
timestamps = create_timestamps(date, time_step)
sun_azimuths, sun_elevations = get_sun_trajectory(latitude, longitude, timestamps)
angles = get_pv_sun_angle(sun_azimuths, sun_elevations, orientation, tilt)
irradiance_scaling = calc_irradiance_scaling(sun_elevations)
power_ratio = calc_power_ratio(angles, peak_power)

In [164]:
plt.scatter(sun_azimuths, angles[0], label="Panel 1")
plt.scatter(sun_azimuths, angles[1], label="Panel 2")
plt.scatter(sun_azimuths, power_ratio / 50, label="Power ratio")
plt.scatter(sun_azimuths, power_ratio * irradiance_scaling / 50, label="Scaled power ratio")
plt.xlabel("Azimuth in degrees")
plt.ylabel("Angle in degrees")
titletext = (
        "Data for Lat: "
        + str(latitude)
        + " and Lon: "
        + str(longitude)
        + " on "
        + str(date.date())
    )
plt.title(titletext)
plt.legend()
plt.show()